# Parsing data

In [1]:
import pandas as pd

df = pd.read_excel('./data.xlsx')
df['description'] = df['description'].str.replace('_x000D_', '')

df.head()

,sku,category,name,description,price,stock,url
0,SNGRGRSNRGR156582452764,beauty,"Gillette Skinguard Cartridges, 4s",<UL>\n\t<LI><STRONG>CHAT WITH US TO CONFIRM ST...,1697,9,NaN
1,SNGRGRSNRGR156582382763,beauty,"Dial Gold Antibacterial Hydrating Hand Soap, 1...",<UL>\n\t<LI><STRONG>CHAT WITH US TO CONFIRM ST...,1397,9,NaN
2,SNGRGRSNRGR156582142781,beauty,Crest Kid's Anticavity Fluoride Rinse Strawber...,<UL>\n\t<LI><STRONG>CHAT WITH US TO CONFIRM ST...,798,9,NaN
3,SNGRGRSNRGR156582692846,beauty,"Old Spice Timber Bar Soap, 6 bars 846 g.",<UL>\n\t<LI><STRONG>CHAT WITH US TO CONFIRM ST...,554,9,NaN
4,SNGRGRPQEGR156582452874A,beauty,Air Wick VIP Pre-poop Toilet Spray Fresh Model...,<UL>\n\t<LI><STRONG>CHAT WITH US TO CONFIRM ST...,654,9,NaN


## Check column types

In [2]:
df.dtypes

sku             object
category        object
name            object
description     object
price            int64
stock            int64
url            float64
dtype: object

In [3]:
df['sku'] = df['sku'].astype('string')
df['category'] = df['category'].astype('string')
df['name'] = df['name'].astype('string')
df['description'] = df['description'].astype('string')
df['url'] = df['url'].astype('string')

In [4]:
type(df['url'][0])

pandas._libs.missing.NAType

In [5]:
df.dtypes

sku            string[python]
category       string[python]
name           string[python]
description    string[python]
price                   int64
stock                   int64
url            string[python]
dtype: object

## Check SKU uniqueness


In [6]:
row_ct, col_ct = df.shape
sku_ct, = df['sku'].shape
unique_sku_ct, = df['sku'].unique().shape

items = (row_ct, sku_ct, unique_sku_ct)
are_all_same_ct = all(item == items[0] for item in items)

(are_all_same_ct, *items)


(True, 40, 40, 40)

## Set SKU as index

In [7]:
df.head()

,sku,category,name,description,price,stock,url
0,SNGRGRSNRGR156582452764,beauty,"Gillette Skinguard Cartridges, 4s",<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,1697,9,<NA>
1,SNGRGRSNRGR156582382763,beauty,"Dial Gold Antibacterial Hydrating Hand Soap, 1...",<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,1397,9,<NA>
2,SNGRGRSNRGR156582142781,beauty,Crest Kid's Anticavity Fluoride Rinse Strawber...,<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,798,9,<NA>
3,SNGRGRSNRGR156582692846,beauty,"Old Spice Timber Bar Soap, 6 bars 846 g.",<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,554,9,<NA>
4,SNGRGRPQEGR156582452874A,beauty,Air Wick VIP Pre-poop Toilet Spray Fresh Model...,<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,654,9,<NA>


In [8]:
df = df.set_index('sku')

df.head()

,category,name,description,price,stock,url
sku,,,,,,
SNGRGRSNRGR156582452764,beauty,"Gillette Skinguard Cartridges, 4s",<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,1697,9,<NA>
SNGRGRSNRGR156582382763,beauty,"Dial Gold Antibacterial Hydrating Hand Soap, 1...",<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,1397,9,<NA>
SNGRGRSNRGR156582142781,beauty,Crest Kid's Anticavity Fluoride Rinse Strawber...,<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,798,9,<NA>
SNGRGRSNRGR156582692846,beauty,"Old Spice Timber Bar Soap, 6 bars 846 g.",<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,554,9,<NA>
SNGRGRPQEGR156582452874A,beauty,Air Wick VIP Pre-poop Toilet Spray Fresh Model...,<UL> <LI><STRONG>CHAT WITH US TO CONFIRM STOC...,654,9,<NA>


## Parse descriptions


In [9]:
from bs4 import BeautifulSoup
import json

def is_allowed_text(text: str) -> bool:
    if text.startswith('CHAT WITH US'):
        return False
    if text.startswith('EXPIRATION DATE'):
        return False
    if text.startswith('EXPIRY DATE'):
        return False

    return True

def extract_li_texts(markup: str) -> list[str]:
    soup = BeautifulSoup(markup, 'lxml')
    return [string for string in soup.stripped_strings if is_allowed_text(string)]


parsed_desc = [
    json.dumps(
        extract_li_texts(markup)
    )
    for markup in df['description']
]

parsed_desc[:5]


['["Minimizes blade contact with sensitive skin", "Designed for men with skin irritation, razor bumps, and razor burn", "Lubrication before and after the blades for glide and comfort", "SkinGuard blade refills fit all SkinGuard razor handles"]',
 '["Provides antibacterial effectiveness and moisturizing care", "Pleasant, clean aroma", "Liquid soap is convenient and rinses clean"]',
 '["Mouthwash reaches where brushing may miss", "Alcohol-free", "Anticavity fluoride mouthwash", "Fun Strawberry Rush flavor enjoyed by kids", "Designed for kids 6 and up"]',
 '["A smell your body can carve initials into and chop down for shelter", "With fresher scents of Timber and Mint", "Devastates odor"]',
 '["KEEP NASTY SMELLS IN YOUR BOWL: Creates a layer that traps embarrassing odors in the bowl before they escape", "LASTS UP TO 300 SPRAYS: For 100 uses per bottle", "CONTAINS ESSENTIAL OILS: Releases a pleasant fragrance in the air", "MAKES A GREAT GIFT: Friends and family will love it"]']

In [10]:
df['description'] = parsed_desc

df.head()

,category,name,description,price,stock,url
sku,,,,,,
SNGRGRSNRGR156582452764,beauty,"Gillette Skinguard Cartridges, 4s","[""Minimizes blade contact with sensitive skin""...",1697,9,<NA>
SNGRGRSNRGR156582382763,beauty,"Dial Gold Antibacterial Hydrating Hand Soap, 1...","[""Provides antibacterial effectiveness and moi...",1397,9,<NA>
SNGRGRSNRGR156582142781,beauty,Crest Kid's Anticavity Fluoride Rinse Strawber...,"[""Mouthwash reaches where brushing may miss"", ...",798,9,<NA>
SNGRGRSNRGR156582692846,beauty,"Old Spice Timber Bar Soap, 6 bars 846 g.","[""A smell your body can carve initials into an...",554,9,<NA>
SNGRGRPQEGR156582452874A,beauty,Air Wick VIP Pre-poop Toilet Spray Fresh Model...,"[""KEEP NASTY SMELLS IN YOUR BOWL: Creates a la...",654,9,<NA>


## Save as SQLite db

In [11]:
df.head()

,category,name,description,price,stock,url
sku,,,,,,
SNGRGRSNRGR156582452764,beauty,"Gillette Skinguard Cartridges, 4s","[""Minimizes blade contact with sensitive skin""...",1697,9,<NA>
SNGRGRSNRGR156582382763,beauty,"Dial Gold Antibacterial Hydrating Hand Soap, 1...","[""Provides antibacterial effectiveness and moi...",1397,9,<NA>
SNGRGRSNRGR156582142781,beauty,Crest Kid's Anticavity Fluoride Rinse Strawber...,"[""Mouthwash reaches where brushing may miss"", ...",798,9,<NA>
SNGRGRSNRGR156582692846,beauty,"Old Spice Timber Bar Soap, 6 bars 846 g.","[""A smell your body can carve initials into an...",554,9,<NA>
SNGRGRPQEGR156582452874A,beauty,Air Wick VIP Pre-poop Toilet Spray Fresh Model...,"[""KEEP NASTY SMELLS IN YOUR BOWL: Creates a la...",654,9,<NA>


In [29]:
from pathlib import Path
from contextlib import contextmanager
from sqlalchemy import MetaData, Table, Column, Integer, Text, create_engine

filename = 'data.sqlite'
table_name = 'products'

table = Table(
    table_name, MetaData(),
    Column('sku', Text, primary_key=True),
    Column('category', Text),
    Column('name', Text),
    Column('description', Text),
    Column('price', Integer),
    Column('stock', Integer),
    Column('url', Text),
)

@contextmanager
def engine_context():
    '''https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager'''

    engine = create_engine(f'sqlite:///{filename}', echo=False)
    try:
        yield engine
    finally:
        engine.dispose()

Path(filename).unlink(missing_ok=True)  # Delete file first
with engine_context() as engine:
    table.create(engine, checkfirst=True)

    with engine.begin() as con:
        df.to_sql(table_name, con, if_exists='append')